02　TF-IDFを試す
==============

* `TF-IDF`は、単語の出現頻度に定数を掛け算して得られる特徴量

    * したがって、これは`特徴量スケーリング`と言い換えることもできる
    
    * 実際にどの程度昨日しているのか、単純なテキスト分類のタスクでスケーリングされた特徴量と、スケーリングされていない特徴量の性能を比較する

* ここでは、Yelpでのレビューをデータセットとして利用する

    * Yelpチャレンジ第6ラウンドのデータセットには、米国6都市でのレビューが60万件数含まれている

In [2]:
# エラーが発生しているが、専攻分野で必要でない部分なので省略する
import json
import pandas as pd

# Yelpのビジネスデータを読み込み
with open('/Users/kunii.sotaro/Downloads/business.json') as biz_f:
    biz_df = pd.DataFrame([json.loads(x) for x in biz_f.readlines()])

# Yelpのレビューデータを読み込み
with open('/Users/kunii.sotaro/Downloads/review.json') as review_file:
    review_df = pd.DataFrame([json.loads(x) for x in review_file.readlines()])

# YelpのビジネスデータからcategoriesがNightlife（ナイトライフ@<fn>{nightlife}）またはRestaurants(レストラン)のデータを取り出し
filter_func = lambda x:  len(set(x) &  set(['Nightlife', 'Restaurants'])) > 0
twobiz = biz_df[biz_df['categories'].apply(filter_func)]

# 取り出した2つのカテゴリのYelpのビジネスデータとYelpのレビューデータを結合する
twobiz_reviews = twobiz.merge(review_df, on='business_id', how='inner')

# 必要ない特徴量を排除
twobiz_reviews = twobiz_reviews[['business_id', 'name', 'stars_y', 'text', 'categories']]

# target列を作成。categoriesがNightlifeの時はTrue、それ以外の場合はFalse
twobiz_reviews['target'] =  twobiz_reviews['categories'].apply(set(['Nightlife']).issubset)

twobiz

KeyboardInterrupt: 

## 1. クラス分類用のデータセット作成

* レビューを使って、レストランかナイトライフかどうかを分類する

* 学習時間を短縮するために、一部のデータを使う

    * ここでは、2つのカテゴリでレビュー数が大きく異なっている
    
    * このようなデータえっとを、`クラス不均衡データ`と言う
    
    * このデータセットをそのままモデリングすると問題が起き、より大きなクラスに当てはまるように作られてしまう
    
* 今回は、両方のクラスに多くのデータがあるので、`ダウンサンプリング`でこの問題を解決する

    * これは、レコード数が大きいクラスを、レコード数が少ないクラスとほぼ同じサイズになるようにサンプリングして、データを減らす方法

1. ナイトライフに対するレビューの10%とレストランに対するレビューの2.1%をランダムサンプリングする

1. このデータセットの70%を学習データに、30%をテストデータになるようにデータを分離する

1. 学習データには46,924個のユニークな単語が含まれており、これがBag-of-Wordsにおける特徴量の数になる

In [ ]:
from sklearn.model_selection import train_test_split

# サンプリングしてクラス均衡を是正したデータセットを作成
nightlife = twobiz_reviews[twobiz_reviews['categories'].apply(set(['Nightlife']).issubset)]
restaurants = twobiz_reviews[twobiz_reviews['categories'].apply(set(['Restaurants']).issubset)]

nightlife_subset = nightlife.sample(frac=0.1, random_state=123)
restaurant_subset = restaurants.sample(frac=0.021, random_state=123)
combined = pd.concat([nightlife_subset, restaurant_subset])

# 学習データとテストデータに分割
training_data, test_data = train_test_split(combined, test_size=0.3, random_state=123)
training_data.shape

In [ ]:
test_data.shape

## 2. TF-IDF変換を用いたBag-of-Wordsのスケーリング

* 線形クラス分類における`Bag-of-Words`、`TF-IDF`、$l^2$正規化の効果を実験により比較する

    * `TF-IDF`に$l^2$正規化を行なった際の結果は、$l^2$正規化を単体で使う場合と同じ
    
    * 従って、`Bag-of-Words`、`TF-IDF`、Bag-of-Wordsの$l^2$正規化の3つについての実験が必要になる

* 以下の例では、scikit-learnの`CountVectorizer`を使用してレビューテキストを`Bag-of-Words`に変換する

    * 全てのテキストを特徴量化する手法は、テキストをトークン(単語)のリストに変換する機能であるトークナイザに依存する
    
    * 全てのテキストを特徴量化する手法は、テキストをトークン(単語)のリストに変換する機能であるトークナイザに依存する
    
* この例では、scikit-learnのデフォルトのトークン化パターンは、2つ以上の英数字の連続を探す

    * 句読点は、トークンを分割する区切り文字となる

In [ ]:
from sklearn.feature_extraction import text

# レビューをBag-of-Wordsで表す
bow_transform = text.CountVectorizer()
X_tr_bow = bow_transform.fit_transform(training_data['text'])
X_te_bow = bow_transform.transform(test_data['text'])
len(bow_transform.vocabulary_)

In [ ]:
from sklearn.preprocessing import normalize

y_tr = training_data['target']
y_te = test_data['target']

# Bag-of-Words行列からTF-IDFを作成
tfidf_trfm = text.TfidfTransformer(norm=None)
X_tr_tfidf = tfidf_trfm.fit_transform(X_tr_bow)
X_te_tfidf = tfidf_trfm.transform(X_te_bow)

#  Bag-of-WordsのL2正規化
X_tr_l2 = normalize(X_tr_bow, norm='l2', axis=0)
X_te_l2 = normalize(X_te_bow, norm='l2', axis=0)

> **テストデータにおける特徴量スケーリング**
>
> 特徴量をスケーリングする際のポイントとして、`テストデータの平均`、`分散`、`文書頻度`、$l^2$ノルムなど、実際にはわからない可能性が高い特徴量の統計を使う必要がある
>
> `TF-IDF`を計算するには、**学習データ**で逆文書頻度を計算して、学習データおよびテストデータの両方をスケーリングする
>
> scikit-learnでは、学習データで特徴量変換をすると、関連する統計量が保存される
> 
> テストデータの特徴量に対しても、学習データと同じ統計量で変換することができる

* 学習データでテストデータのスケーリングをすると、不自然な結果となる可能性がある

    * テストデータに対してMin-Maxスケーリングをすると、最大1で最小10に変換されない
    
    * これは、$l^2$ノルム、`平均`、`分散`が学習データとテストデータで異なるため
    
* 一般的な解決方法は、新しい単語をテストデータから取り除く

## 3. ロジスティック回帰によるクラス分類

* `ロジスティック回帰`：シンプルな線形分類器であり、そのシンプルさゆえに最初に試す分類器として適している

    * 重みのついた特徴量を足し合わせた結果を、**シグモイド関数**に渡す
    
    * この関数は、実数$x$を0から1の連続値に変換する
    
        * $w$：傾き
    
        * $b$：切片項、関数出力がどこで中間点0.5になるかを表す
    


* `ロジスティック回帰`では、シグモイド関数の出力が0.5より大きい場合は正のクラスを、そうでない場合は負のクラスとちえ予測する

    * クラスの境界地点がどこになるか、入力値の変化の影響度合いをパラメータ$w$と$b$で決定する

![sigmoid関数](./images/sigmoid関数.png)

* 次に、3パターンのデータセットでロジスティック回帰を作り挙動を確認する

In [ ]:
from sklearn.linear_model import LogisticRegression

def simple_logistic_classify(X_tr, y_tr, X_test, y_test, description, _C=1.0):
### ロジスティック回帰で学習し、テストデータでの予測結果を得る関数
    m = LogisticRegression(solver='liblinear', C=_C).fit(X_tr, y_tr)
    s = m.score(X_test, y_test)
    print ('Test score with', description, 'features:', s)
    return m

m1 = simple_logistic_classify(X_tr_bow, y_tr, X_te_bow, y_te, 'bow')
m2 = simple_logistic_classify(X_tr_l2, y_tr, X_te_l2, y_te, 'l2-normalized')
m3 = simple_logistic_classify(X_tr_tfidf, y_tr, X_te_tfidf, y_te, 'tf-idf')

* Bag-of-Wordsを用いるモデルが比較対象の中で最も精度が高くなっている

    * この結果は、モデルのチューニングが不十分のため起きた現象で、モデルを比較する時に犯す典型的なミスの1つ

## 4. 正則化によるロジスティック回帰のチューニング

![4-2-1](./images/4-2-1.jpg)

![4-2-2](./images/4-2-2.jpg)

![4-2-3](./images/4-2-3.jpg)

![4-2-4](./images/4-2-4.jpg)

In [ ]:
from sklearn.model_selection import GridSearchCV

# 探索範囲を指定して、5分割でグリッドサーチを実行します
param_grid_ = {'C': [1e-5, 1e-3, 1e-1, 1e0, 1e1, 1e2]}

# Bag-of-Wordsでの分類器をチューニング
bow_search = GridSearchCV(LogisticRegression(solver='liblinear'), cv=5, param_grid=param_grid_, return_train_score=True)
bow_search.fit(X_tr_bow, y_tr)

# L2正規化単語ベクトルでの分類器をチューニング
l2_search = GridSearchCV(LogisticRegression(solver='liblinear'), cv=5, param_grid=param_grid_, return_train_score=True)
l2_search.fit(X_tr_l2, y_tr)

# TF-IDFでの分類器をチューニング
tfidf_search = GridSearchCV(LogisticRegression(solver='liblinear'), cv=5, param_grid=param_grid_, return_train_score=True)
tfidf_search.fit(X_tr_tfidf, y_tr)

# グリッドサーチにおける出力を見て、挙動を確認します
bow_search.cv_results_

In [ ]:
# クロスバリデーションの結果を箱ひげ図でプロットする
# 分類器のパフォーマンスを可視化して比較する
search_results = pd.DataFrame.from_dict({
    'bow': bow_search.cv_results_['mean_test_score'],
    'tfidf': tfidf_search.cv_results_['mean_test_score'],
    'l2': l2_search.cv_results_['mean_test_score']
})

# matplotlibでグラフを描く。ここでSeabornはグラフの見た目を整える為に用いている。
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")
ax = sns.boxplot(data=search_results, width=0.4)
ax.set_ylabel('Accuracy', size=14)
ax.tick_params(labelsize=14)

In [ ]:
# クロスバリデーションで得られた最適なハイパーパラメータと学習用データ全てを用いて最終的なモデルを学習し、そのモデルを用いて検証用データにおける精度を算出する
m1 = simple_logistic_classify(X_tr_bow, y_tr, X_te_bow, y_te, 'bow',  _C=bow_search.best_params_['C'])
m2 = simple_logistic_classify(X_tr_l2, y_tr, X_te_l2, y_te, 'l2-normalized',  _C=l2_search.best_params_['C'])
m3 = simple_logistic_classify(X_tr_tfidf, y_tr, X_te_tfidf, y_te, 'tf-idf',  _C=tfidf_search.best_params_['C'])